In [1]:
import pandas as pd
import numpy as np
from scipy.io import wavfile
from scipy.signal import stft

In [2]:
df_dev = pd.read_csv('development.csv', index_col=0)
print('The shape of development dataset is ', df_dev.shape)
df_dev.head()

The shape of development dataset is  (9854, 9)


,path,speakerId,action,object,Self-reported fluency level,First Language spoken,Current language used for work/school,gender,ageRange
Id,,,,,,,,,
0,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0a312...,2BqVo8kVB2Skwgyb,change language,none,advanced,English (United States),English (United States),female,22-40
1,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0ee42...,2BqVo8kVB2Skwgyb,activate,music,advanced,English (United States),English (United States),female,22-40
2,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/1d9f3...,2BqVo8kVB2Skwgyb,deactivate,lights,advanced,English (United States),English (United States),female,22-40
3,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/269fc...,2BqVo8kVB2Skwgyb,increase,volume,advanced,English (United States),English (United States),female,22-40
4,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/5bbda...,2BqVo8kVB2Skwgyb,increase,volume,advanced,English (United States),English (United States),female,22-40


In [3]:
df_dev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9854 entries, 0 to 9853
Data columns (total 9 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   path                                   9854 non-null   object
 1   speakerId                              9854 non-null   object
 2   action                                 9854 non-null   object
 3   object                                 9854 non-null   object
 4   Self-reported fluency level            9854 non-null   object
 5   First Language spoken                  9854 non-null   object
 6   Current language used for work/school  9854 non-null   object
 7   gender                                 9854 non-null   object
 8   ageRange                               9854 non-null   object
dtypes: object(9)
memory usage: 769.8+ KB


In [4]:
df_dev.isna().sum()

path                                     0
speakerId                                0
action                                   0
object                                   0
Self-reported fluency level              0
First Language spoken                    0
Current language used for work/school    0
gender                                   0
ageRange                                 0
dtype: int64

In [5]:
def read_wavfile(path, spilitter):
    
    sample_rate, data = wavfile.read( path.split(spilitter)[1] )
    channel = data.ndim
    
    if channel == 1:
        no_sample = data.shape[0]
        length = data.shape[0]/sample_rate
    elif channel == 2:
        no_sample = data.shape[1]
        length = data.shape[1]/sample_rate
        
    return sample_rate, length, no_sample, channel, data

In [6]:
df_dev[['sample_rate', 'length', 'no_sample', 'channel', 'data']] = df_dev.apply(lambda r: read_wavfile(r['path'], 'dsl_data/'), axis=1, result_type='expand')

In [7]:
df_dev.head()

,path,speakerId,action,object,Self-reported fluency level,First Language spoken,Current language used for work/school,gender,ageRange,sample_rate,length,no_sample,channel,data
Id,,,,,,,,,,,,,,
0,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0a312...,2BqVo8kVB2Skwgyb,change language,none,advanced,English (United States),English (United States),female,22-40,16000,1.857625,29722,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/0ee42...,2BqVo8kVB2Skwgyb,activate,music,advanced,English (United States),English (United States),female,22-40,16000,1.393188,22291,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/1d9f3...,2BqVo8kVB2Skwgyb,deactivate,lights,advanced,English (United States),English (United States),female,22-40,16000,1.950500,31208,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/269fc...,2BqVo8kVB2Skwgyb,increase,volume,advanced,English (United States),English (United States),female,22-40,16000,1.764687,28235,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,dsl_data/audio/speakers/2BqVo8kVB2Skwgyb/5bbda...,2BqVo8kVB2Skwgyb,increase,volume,advanced,English (United States),English (United States),female,22-40,16000,2.414875,38638,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
df_dev['class'] = df_dev['action'] + df_dev['object'].str.replace('none','')

In [9]:
df_dev.drop(['path', 'action', 'object'], axis=1, inplace=True)

In [10]:
df_dev.head()

,speakerId,Self-reported fluency level,First Language spoken,Current language used for work/school,gender,ageRange,sample_rate,length,no_sample,channel,data,class
Id,,,,,,,,,,,,
0,2BqVo8kVB2Skwgyb,advanced,English (United States),English (United States),female,22-40,16000,1.857625,29722,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",change language
1,2BqVo8kVB2Skwgyb,advanced,English (United States),English (United States),female,22-40,16000,1.393188,22291,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",activatemusic
2,2BqVo8kVB2Skwgyb,advanced,English (United States),English (United States),female,22-40,16000,1.950500,31208,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",deactivatelights
3,2BqVo8kVB2Skwgyb,advanced,English (United States),English (United States),female,22-40,16000,1.764687,28235,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",increasevolume
4,2BqVo8kVB2Skwgyb,advanced,English (United States),English (United States),female,22-40,16000,2.414875,38638,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",increasevolume


In [11]:
df_dev['channel'].unique()

array([1], dtype=int64)

In [12]:
df_dev.drop('channel', axis=1, inplace=True)

In [ ]:
df_dev.head()

,speakerId,Self-reported fluency level,First Language spoken,Current language used for work/school,gender,ageRange,sample_rate,length,no_sample,data,class
Id,,,,,,,,,,,
0,2BqVo8kVB2Skwgyb,advanced,English (United States),English (United States),female,22-40,16000,1.857625,29722,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",change language
1,2BqVo8kVB2Skwgyb,advanced,English (United States),English (United States),female,22-40,16000,1.393188,22291,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",activatemusic
2,2BqVo8kVB2Skwgyb,advanced,English (United States),English (United States),female,22-40,16000,1.950500,31208,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",deactivatelights
3,2BqVo8kVB2Skwgyb,advanced,English (United States),English (United States),female,22-40,16000,1.764687,28235,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",increasevolume
4,2BqVo8kVB2Skwgyb,advanced,English (United States),English (United States),female,22-40,16000,2.414875,38638,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",increasevolume


In [61]:
def cal_mean_std_blocks(wav_file_info, n, m):
    
    _, _, zxx = stft(wav_file_info)
    zxx = abs(zxx)
    zxx = zxx.T
    
    means = []
    stds = []
    
    for t_segment in np.array_split(zxx, n):
        for f_segment in np.array_split(t_segment, m):
            means.append(np.mean(f_segment))
            stds.append(np.std(f_segment))
            
    return means, stds

In [62]:
no_time_split = 10
no_freq_split = 10
df_dev[['bls_mean', 'bls_std']] = df_dev.apply(lambda r: cal_mean_std_blocks(r['data'], no_time_split, no_freq_split), axis=1, result_type='expand')

C:\Users\Iman\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Iman\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
C:\Users\Iman\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Iman\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\Iman\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcou